In [1]:
import re
import os
import unicodedata
import shutil
from pathlib import Path
from typing import List, Tuple, Optional
from google.colab import files
import zipfile

In [2]:
# Основной код парсера

# Специальные символы, допустимые в начале статьи
ALLOWED_PREFIX_SYMBOLS = {
    '\uf04b',  # новое слово
    '\uf04a',  # слово, выпавшее из употребления
    '\uf048',  # слово, выходящее из употребления
    '\uf044',  # новое слово, в XVIII в. вышедшее из употребления
    '\u22c6',  # ⋆ - слово, расширившее употребление
}

# Расширенный список всех Private Use Area символов
PUA_RANGE = range(0xE000, 0xF900)


def is_allowed_prefix_char(char: str) -> bool:
    """Проверяет, допустим ли символ в префиксе перед заголовком статьи."""
    if char.isspace():
        return True
    if char in ALLOWED_PREFIX_SYMBOLS:
        return True
    if ord(char) in PUA_RANGE:
        return True
    return False


def is_uppercase_heading(text: str) -> bool:
    """Проверяет, является ли текст заголовком статьи (все буквы заглавные)."""
    text_no_accents = text.replace('\u0301', '')

    #Римские цифры
    roman_pattern = r'^[IVXLCDM]+[\.\,\s]*$'
    if re.match(roman_pattern, text_no_accents.strip()):
        return False

    # Извлекаем только буквы
    letters = [c for c in text_no_accents if c.isalpha()]

    if not letters:
        return False

    # Одиночная буква разделитель главы
    if len(letters) == 1:
        return False

    # Все буквы должны быть заглавными
    for letter in letters:
        if letter.islower():
            return False

    return True


def extract_headword(text: str) -> str:
    """Извлекает слово-заголовок из текста для имени файла."""
    # Сохраняем все символы, включая знак ударения и старые русские буквы
    # Убираем только запятые и пробелы в конце
    result = text.rstrip(', ')
    return result if result else "UNKNOWN"


def parse_xml_entries(xml_content: str) -> List[Tuple[str, str, str]]:
    """Разбирает XML на отдельные статьи."""
    entries = []

    p_pattern = re.compile(r'<p>(.*?)</p>', re.DOTALL)

    entry_start_pattern = re.compile(
        r'^([\s\uf044\uf048\uf04a\uf04b\u22c6]*)'
        r'<hi\s+rendition="simple:bold">'
        r'([^<]+)'
        r'</hi>',
        re.UNICODE
    )

    all_paragraphs = list(p_pattern.finditer(xml_content))

    current_entry_parts = []
    current_headword = None
    current_prefix = ""

    for i, match in enumerate(all_paragraphs):
        p_content = match.group(1)
        full_p = match.group(0)

        entry_match = entry_start_pattern.match(p_content)

        is_new_entry = False
        if entry_match:
            prefix = entry_match.group(1)
            heading_text = entry_match.group(2)

            prefix_valid = all(is_allowed_prefix_char(c) for c in prefix)

            if prefix_valid and is_uppercase_heading(heading_text):
                is_new_entry = True
                new_prefix = prefix.strip()
                new_headword = extract_headword(heading_text)

        if is_new_entry:
            if current_headword and current_entry_parts:
                entry_content = '\n'.join(current_entry_parts)
                entries.append((current_headword, current_prefix, entry_content))

            current_headword = new_headword
            current_prefix = new_prefix
            current_entry_parts = [full_p]
        else:
            if current_headword:
                current_entry_parts.append(full_p)

    if current_headword and current_entry_parts:
        entry_content = '\n'.join(current_entry_parts)
        entries.append((current_headword, current_prefix, entry_content))

    return entries


def convert_tei_to_html_span(content: str) -> str:
    """Конвертирует TEI разметку в HTML с inline стилями."""
    # Bold
    content = re.sub(
        r'<hi\s+rendition="simple:bold">(.*?)</hi>',
        "<span style=\"font-family:'Times New Roman'; font-weight:bold\">\\1</span>",
        content,
        flags=re.DOTALL
    )

    # Italic
    content = re.sub(
        r'<hi\s+rendition="simple:italic">(.*?)</hi>',
        "<span style=\"font-family:'Times New Roman'; font-style:italic\">\\1</span>",
        content,
        flags=re.DOTALL
    )

    # Superscript
    content = re.sub(
        r'<hi\s+rendition="simple:superscript">(.*?)</hi>',
        "<span style=\"line-height:115%; font-family:'Times New Roman'; font-size:9.33pt; vertical-align:super\">\\1</span>",
        content,
        flags=re.DOTALL
    )

    # Letter spacing
    content = re.sub(
        r'<hi\s+rendition="simple:letterspace">(.*?)</hi>',
        "<span style=\"font-family:'Times New Roman'; letter-spacing:2pt\">\\1</span>",
        content,
        flags=re.DOTALL
    )

    # Subscript
    content = re.sub(
        r'<hi\s+rendition="simple:subscript">(.*?)</hi>',
        "<span style=\"line-height:115%; font-family:'Times New Roman'; font-size:9.33pt; vertical-align:sub\">\\1</span>",
        content,
        flags=re.DOTALL
    )

    return content


def wrap_plain_text(content: str) -> str:
    """Оборачивает простой текст в span с Times New Roman."""
    result = []
    i = 0

    while i < len(content):
        if content[i] == '<':
            end = content.find('>', i)
            if end == -1:
                result.append(content[i:])
                break

            tag = content[i:end+1]

            if tag.startswith('<span'):
                close_idx = content.find('</span>', end)
                if close_idx != -1:
                    result.append(content[i:close_idx + 7])
                    i = close_idx + 7
                    continue

            result.append(tag)
            i = end + 1
        else:
            next_tag = content.find('<', i)
            if next_tag == -1:
                text = content[i:]
                i = len(content)
            else:
                text = content[i:next_tag]
                i = next_tag

            if text:
                has_pua = any(0xE000 <= ord(c) <= 0xF8FF for c in text)

                if has_pua and len(text.strip()) <= 2:
                    result.append(f'<span style="font-family:Sym0">{text}</span>')
                elif text.strip():
                    result.append(f"<span style=\"font-family:'Times New Roman'\">{text}</span>")
                else:
                    if text:
                        result.append(f"<span style=\"font-family:'Times New Roman'\">{text}</span>")

    return ''.join(result)


def create_html_document(content: str, headword: str) -> str:
    """Создает полный HTML документ для статьи."""
    html_content = convert_tei_to_html_span(content)
    html_content = wrap_plain_text(html_content)

    html_content = re.sub(
        r'<p>(.*?)</p>',
        r'<p style="margin-top:0pt; margin-bottom:10pt; line-height:115%; font-size:14pt">\1</p>',
        html_content,
        flags=re.DOTALL
    )

    html_template = '''<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="text/css" http-equiv="Content-Style-Type"/><meta content="Dictionary Parser" name="generator"/><title>{title}</title></head><body style="line-height:115%; font-family:Cambria; font-size:11pt"><div><div style="-aw-headerfooter-type:header-primary; clear:both"><p style="margin-top:0pt; margin-bottom:10pt"><span style="height:0pt; display:block; position:absolute; z-index:-65537"></span></p></div><p style="margin-top:0pt; margin-bottom:10pt; line-height:115%; font-size:12pt"></p>{content}<p style="margin-top:0pt; margin-bottom:10pt; line-height:115%; font-size:14pt"><span style="font-family:'Times New Roman'; -aw-import:spaces">  </span></p><div style="-aw-headerfooter-type:footer-primary; clear:both"><p style="margin-top:0pt; margin-bottom:10pt; line-height:115%; font-size:12pt"></p></div></div></body></html>'''

    return html_template.format(title=headword, content=html_content)


def sanitize_filename(name: str) -> str:
    """Создает безопасное имя файла."""
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        name = name.replace(char, '_')
    return name


def process_xml_file(input_path: str, output_dir: str) -> dict:
    """Обрабатывает XML файл и создает HTML файлы для каждой статьи."""
    # Создаем выходную директорию
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    print(f"Создана директория: {output_dir}")

    # Читаем XML файл
    print(f"Чтение XML файла...")
    with open(input_path, 'r', encoding='utf-8') as f:
        xml_content = f.read()
    print(f"Прочитано {len(xml_content)} символов")

    # Парсим статьи
    print(f"Парсинг статей...")
    entries = parse_xml_entries(xml_content)
    print(f"Найдено {len(entries)} статей")

    stats = {
        'total_entries': len(entries),
        'files_created': 0,
        'errors': [],
        'entries': []
    }

    # Словарь для отслеживания дубликатов имен
    name_counts = {}

    print(f"Сохранение HTML файлов...")
    for idx, (headword, prefix, content) in enumerate(entries, 1):
        try:
            # Формируем имя файла
            base_name = sanitize_filename(headword)

            # Обрабатываем дубликаты
            if base_name in name_counts:
                name_counts[base_name] += 1
                filename = f"{base_name}_{name_counts[base_name]}.html"
            else:
                name_counts[base_name] = 0
                filename = f"{base_name}.html"

            # Создаем HTML документ
            html_doc = create_html_document(content, headword)

            # Сохраняем файл
            output_file = output_path / filename
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(html_doc)

            stats['files_created'] += 1
            stats['entries'].append({
                'headword': headword,
                'prefix': prefix if prefix else None,
                'filename': filename
            })

            if idx % 100 == 0:
                print(f"   Обработано: {idx}/{len(entries)}")

        except Exception as e:
            error_msg = f"Error processing '{headword}': {str(e)}"
            stats['errors'].append(error_msg)
            print(f"!УВАГА!  {error_msg}")

    print(f"Сохранено {stats['files_created']} файлов")
    return stats

print("Функции парсера загружены")

Функции парсера загружены


In [3]:
# Путь к XML
INPUT_XML_FILE = '/content/Выпуск_3_Век_Воздувать_Отправка_fixed_TEI.xml' # ------- МЕНЯТЬ ИМЯ ЗДЕСЬ
OUTPUT_DIR = '/content/dictionary_output'

# Проверяем существование входного файла
if not os.path.exists(INPUT_XML_FILE):
    print(f"\n ОШИБКА: Файл не найден: {INPUT_XML_FILE}")
    print("\nПожалуйста:")
    print("1. Загрузите ваш XML файл в Colab (Files → Upload)")
    print("2. Укажите правильный путь в переменной INPUT_XML_FILE")
    print("   Пример: INPUT_XML_FILE = '/content/dictionary.xml'")
else:
    print(f"\n Входной файл: {INPUT_XML_FILE}")
    print(f" Выходная директория: {OUTPUT_DIR}")

    try:
        # Обрабатываем файл
        stats = process_xml_file(INPUT_XML_FILE, OUTPUT_DIR)

        print("ОБРАБОТКА ЗАВЕРШЕНА")
        print(f"Найдено статей: {stats['total_entries']}")
        print(f"Создано файлов: {stats['files_created']}")

        if stats['errors']:
            print(f"\n Ошибки ({len(stats['errors'])}):")
            for error in stats['errors'][:10]:
                print(f"   - {error}")
            if len(stats['errors']) > 10:
                print(f"   ... и ещё {len(stats['errors']) - 10} ошибок")

        # Показываем примеры
        print(f"\n Примеры статей (первые 10):")
        for entry in stats['entries'][:10]:
            prefix_str = f" [{entry['prefix']}]" if entry['prefix'] else ""
            print(f"   {entry['headword']}{prefix_str} → {entry['filename']}")
        if len(stats['entries']) > 10:
            print(f"   ... и ещё {len(stats['entries']) - 10} статей")

        # Проверяем выходную директорию
        if os.path.exists(OUTPUT_DIR):
            files_in_dir = os.listdir(OUTPUT_DIR)
            print(f"\n Проверка выходной директории:")
            print(f"   Путь: {OUTPUT_DIR}")
            print(f"   Файлов: {len(files_in_dir)}")

            if len(files_in_dir) > 0:
                # Создаем архив
                print(f"\n Создание архива...")
                zip_filename = '/content/dictionary_output.zip' # ---- имя архива

                files_added = 0
                with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
                    for file in files_in_dir:
                        file_path = os.path.join(OUTPUT_DIR, file)
                        zipf.write(file_path, file)
                        files_added += 1

                zip_size = os.path.getsize(zip_filename)
                print(f"Архив создан: {zip_filename}")
                print(f"Файлов в архиве: {files_added}")
                print(f"Размер архива: {zip_size / 1024:.2f} КБ")

                # Скачивание
                files.download(zip_filename)
            else:
                print(f"\n ВНИМАНИЕ: Директория пуста!")
        else:
            print(f"\n ВНИМАНИЕ: Директория не создана!")

    except Exception as e:
        print(f"\n ОШИБКА ПРИ ОБРАБОТКЕ")
        print("="*60)
        print(f"Описание: {str(e)}")
        print("\nПодробности:")
        import traceback
        print(traceback.format_exc())


 Входной файл: /content/Выпуск_3_Век_Воздувать_Отправка_fixed_TEI.xml
 Выходная директория: /content/dictionary_output
Создана директория: /content/dictionary_output
Чтение XML файла...
Прочитано 2018623 символов
Парсинг статей...
Найдено 1682 статей
Сохранение HTML файлов...
   Обработано: 100/1682
   Обработано: 200/1682
   Обработано: 300/1682
   Обработано: 400/1682
   Обработано: 500/1682
   Обработано: 600/1682
   Обработано: 700/1682
   Обработано: 800/1682
   Обработано: 900/1682
   Обработано: 1000/1682
   Обработано: 1100/1682
   Обработано: 1200/1682
   Обработано: 1300/1682
   Обработано: 1400/1682
   Обработано: 1500/1682
   Обработано: 1600/1682
Сохранено 1682 файлов
ОБРАБОТКА ЗАВЕРШЕНА
Найдено статей: 1682
Создано файлов: 1682

 Примеры статей (первые 10):
   ВѢК → ВѢК.html
   ВѢКОВА́ТЬ. → ВѢКОВА́ТЬ..html
   ВѢКОВѢЧНОСТЬ → ВѢКОВѢЧНОСТЬ.html
   ВѢКОВѢЧНЫЙ → ВѢКОВѢЧНЫЙ.html
   ВѢКОВО́Й → ВѢКОВО́Й.html
   ВѢКОТВОРИТЬ → ВѢКОТВОРИТЬ.html
   ВѢКОЧИСЛЕНИЕ → ВѢКОЧИСЛЕНИЕ.html
 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>